In [1]:
from datasets import load_dataset 
from transformers import AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from torchvision.transforms import CenterCrop,Compose,Normalize,RandomHorizontalFlip,RandomResizedCrop,Resize,ToTensor,ColorJitter

In [2]:
!nvidia-smi

Wed Nov 16 15:47:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [3]:
from datasets import load_metric
metric = load_metric("accuracy")

In [4]:
pretrained = "google/vit-base-patch16-224" 
batch_size = 85 

In [5]:
data_files = {'train': ['../input/intel-image-classification/seg_train/seg_train**'],
             'test': ['../input/intel-image-classification/seg_test/seg_test**']}
dataset = load_dataset("imagefolder", data_files=data_files, cache_dir='./', task='image-classification')

Resolving data files:   0%|          | 0/14034 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3000 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to ./image_folder/default-27e99938fa020a11/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 14034
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 3000
    })
})

In [7]:
dataset["train"].features

{'image': Image(decode=True, id=None),
 'labels': ClassLabel(num_classes=6, names=['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'], id=None)}

In [8]:
labels = dataset["train"].features["labels"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [9]:
label2id

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

In [10]:
feature_extractor = AutoFeatureExtractor.from_pretrained(pretrained)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

In [11]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

train_transforms = Compose([RandomResizedCrop(feature_extractor.size),
                            ToTensor(),
                            normalize])

val_transforms = Compose([Resize(feature_extractor.size),
                        CenterCrop(feature_extractor.size),
                        ToTensor(),
                        normalize])

def preprocess_train(batch):
    batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in batch["image"]]
    return batch

def preprocess_val(batch):
    batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in batch["image"]]
    return batch

In [12]:
train_ds = dataset['train']
val_ds = dataset['test']
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [13]:
val_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=150x150>,
 'labels': 0,
 'pixel_values': tensor([[[-0.2941, -0.2000, -0.1294,  ..., -0.1216, -0.1216, -0.1216],
          [-0.3176, -0.1294,  0.0353,  ..., -0.1451, -0.1451, -0.1451],
          [-0.2784, -0.0431,  0.1765,  ..., -0.1765, -0.1765, -0.1765],
          ...,
          [ 0.5294,  0.5294,  0.5294,  ...,  0.7804,  0.7961,  0.8118],
          [ 0.5216,  0.5216,  0.5216,  ...,  0.7882,  0.8118,  0.8275],
          [ 0.5059,  0.5059,  0.5059,  ...,  0.7961,  0.8196,  0.8353]],
 
         [[-0.0667,  0.0196,  0.0745,  ...,  0.1608,  0.1608,  0.1608],
          [-0.0902,  0.0902,  0.2392,  ...,  0.1608,  0.1608,  0.1608],
          [-0.0588,  0.1686,  0.3725,  ...,  0.1608,  0.1608,  0.1608],
          ...,
          [ 0.6157,  0.6157,  0.6157,  ...,  0.8353,  0.8510,  0.8667],
          [ 0.6157,  0.6157,  0.6157,  ...,  0.8431,  0.8667,  0.8824],
          [ 0.6078,  0.6078,  0.6078,  ...,  0.8510,  0.8745,  0.8902]

In [12]:
model = AutoModelForImageClassification.from_pretrained(
    pretrained, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
args = TrainingArguments(
    pretrained,
    num_train_epochs=7,
    learning_rate=10e-5,
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [16]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [17]:
def collate(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate,
)

In [19]:
train_results = trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14034
  Num Epochs = 7
  Instantaneous batch size per device = 85
  Total train batch size (w. parallel, distributed & accumulation) = 170
  Gradient Accumulation steps = 1
  Total optimization steps = 581
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.149920,0.946667
2,No log,0.145391,0.949333
3,No log,0.122805,0.956000
4,No log,0.133280,0.954000
5,No log,0.139780,0.956667
6,No log,0.136054,0.956333
7,0.157600,0.133015,0.956667


***** Running Evaluation *****
  Num examples = 3000
  Batch size = 170
Saving model checkpoint to google/vit-base-patch16-224/checkpoint-83
Configuration saved in google/vit-base-patch16-224/checkpoint-83/config.json
Model weights saved in google/vit-base-patch16-224/checkpoint-83/pytorch_model.bin
Feature extractor saved in google/vit-base-patch16-224/checkpoint-83/preprocessor_config.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 170
Saving model checkpoint to google/vit-base-patch16-224/checkpoint-166
Configuration saved in google/vit-base-patch16-224/checkpoint-166/config.json
Model weights saved in google/vit-base-patch16-224/checkpoint-166/pytorch_model.bin
Feature extractor saved in g

In [20]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3000
  Batch size = 170
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.13977988064289093,
 'eval_accuracy': 0.9566666666666667,
 'eval_runtime': 32.0639,
 'eval_samples_per_second': 93.563,
 'eval_steps_per_second': 0.561,
 'epoch': 7.0}